My last notebook was running Python 3.6, but H2O isn't compatible with anything above 3.5. I created a virtual env with python 3.5 and H2O/H2O-py installed, and switched to this notebook to make that distinction clear.

In [2]:
import h2o

In [4]:
h2o.init(port=9992)

Checking whether there is an H2O instance running at http://localhost:9992..... not found.
Attempting to start a local H2O server...
; OpenJDK 64-Bit Server VM (Zulu 8.20.0.5-win64) (build 25.121-b15, mixed mode)
  Starting server from C:\Users\Baptiste\Anaconda3\envs\py35\h2o_jar\h2o.jar
  Ice root: C:\Users\Baptiste\AppData\Local\Temp\tmpo7uk7bi_
  JVM stdout: C:\Users\Baptiste\AppData\Local\Temp\tmpo7uk7bi_\h2o_Baptiste_started_from_python.out
  JVM stderr: C:\Users\Baptiste\AppData\Local\Temp\tmpo7uk7bi_\h2o_Baptiste_started_from_python.err
  Server is running at http://127.0.0.1:9992
Connecting to H2O server at http://127.0.0.1:9992... successful.


H2O cluster uptime:,03 secs
H2O cluster version:,3.10.0.9
H2O cluster version age:,10 months and 9 days !!!
H2O cluster name:,H2O_from_python_Baptiste_fpd1lw
H2O cluster total nodes:,1
H2O cluster free memory:,1.757 Gb
H2O cluster total cores:,4
H2O cluster allowed cores:,4
H2O cluster status:,"accepting new members, healthy"
H2O connection url:,http://127.0.0.1:9992
H2O connection proxy:,None


In [18]:
h2o_train_df = h2o.import_file('./train_dataset.csv')
h2O_test_df = h2o.import_file('./test_dataset.csv')

Parse progress: |█████████████████████████████████████████████████████████| 100%
Parse progress: |█████████████████████████████████████████████████████████| 100%


In [15]:
cat_feat=['class', 'edu', 'marital', 'race', 'sex', 'fam_pres',
          'full_part_time', 'tax_filer', 'detailed_fam', 'detailed_house',
         'mig_code_msa', 'country_m','enterp', 'child', 'capital', 'ind_code', 'oc_code']
cont_feat=['age', 'wage', 'cap_gains', 'cap_losses', 'dividends', 'employer', 'weeks_w']
predict=['inc_bin']

In [20]:
for cat_col in cat_feat:
    h2o_train_df[cat_col]=h2o_train_df[cat_col].asfactor()
    h2O_test_df[cat_col]=h2O_test_df[cat_col].asfactor()

In [ ]:
from h2o.estimators.random_forest import H2ORandomForestEstimator
from random import shuffle
from sklearn.metrics import precision_recall_fscore_support
import analytics_functions
#from pyprooff.perf import Togo
 
h2o.__PROGRESS_BAR__=False
h2o.no_progress()

In [ ]:
frtr = h2o.import_file('for_h2o_tr_both.csv')
 
trcol = [x for x in df.columns if x not in ['case_id', 'target', 'fold']]
#71 feats
 
model = H2ORandomForestEstimator(ntrees=200, score_tree_interval=10, keep_cross_validation_predictions=True,
                                  stopping_rounds=3,
                                  stopping_tolerance=0.001,
                                  mtries=25,
                                  col_sample_rate_change_per_level=1.2,
                                  sample_rate=0.62,
                                  col_sample_rate_per_tree=0.2,
                                  max_depth=10,
                                  min_rows=15,
                                  nbins=20,
                                  nbins_cats=200,
                                  min_split_improvement=1e-05,
                                  balance_classes=True)
model.train(x=trcol, y='target', training_frame=frtr, fold_column='fold', max_runtime_secs=300)
 
preds = model.cross_validation_holdout_predictions()
preds_pd = preds.as_data_frame()
preds_list = list(preds_pd.predict)
evaluation = analytics_functions.evaluate_model(dftr.target, preds_list, early_eval=0.25, train_mse=model.mse(train=True))
  
ncols = len(h2o_nonpw.columns)
mt = int(ncols/3)
hyp = {'mtries': [mt], # default; tried 25% less and more, did slightly worse
          'col_sample_rate_change_per_level': [0.8, 1.0, 1.2],
          'sample_rate': [.62],
           'col_sample_rate_per_tree': [0.15, 0.2],
           'max_depth': [4,6,8,10],
           'min_rows': [15],
           'nbins': [20], # 30 was same as 20
           'nbins_cats': [200],
           'min_split_improvement': [1e-05],
           'random_split_points': [False], #[True, False],
           'balance_classes': [True]
          }
 
from itertools import product
conditions = []
for coll in product(hyp['mtries'], hyp['col_sample_rate_change_per_level'],
                    hyp['sample_rate'], hyp['col_sample_rate_per_tree'],
                    hyp['max_depth'], hyp['min_rows'], hyp['nbins'],
                    hyp['nbins_cats'],
                    hyp['min_split_improvement'], hyp['random_split_points'], hyp['balance_classes']):
    conditions.append(coll)
 
results = []
h2o_nonpw.columns
 
warnings.filterwarnings("ignore", module='sklearn')
# togo = Togo(len(conditions), 500, 1, 2)
 
trcol = [x for x in df.columns if x not in ['case_id', 'target', 'fold']]
 
for i in range(len(conditions)):
    print(i)
    shuffle(conditions)
    m1, c2, s3, c4, m5, m6, n7, n8, m9, r10, b11 = conditions[0]
    conditions = conditions[1:]
    for rep in range(3):
        if True:
            model = H2ORandomForestEstimator(ntrees=200, score_tree_interval=10, keep_cross_validation_predictions=True,
                                        stopping_rounds=3, stopping_tolerance=0.001, mtries=m1, col_sample_rate_change_per_level=c2,
                                            sample_rate=s3, col_sample_rate_per_tree=c4, max_depth=m5, min_rows=m6,
                                            nbins=n7, nbins_cats=n8, min_split_improvement=m9,
                                            random_split_points=r10, balance_classes=b11)
            model.train(x=trcol, y='target', training_frame=frtr, fold_column='fold', max_runtime_secs=300)
            preds = model.cross_validation_holdout_predictions()
            preds = preds.as_data_frame()
            preds = list(preds.predict)
            d = evaluate_model(dftr.target, preds, early_eval=0.25, train_mse=model.mse(train=True))
            for name, item in zip(['mtries', 'col_sample_rate_change_per_level',
                                            'sample_rate', 'col_sample_rate_per_tree', 'max_depth', 'min_rows',
                                            'nbins', 'nbins_cats', 'min_split_improvement',
                                            'random_split_points', 'balance_classes'],
                                    "m1, c2, s3, c4, m5, m6, n7, n8, m9, r10, b11".split(", ")):
                exec("d['{}'] = {}".format(name, item))
            d['num_trees'] =  model.scoring_history()['number_of_trees'].iloc[-1]
            d['rep'] = rep
            results.append(d)